In [40]:
#Algorithm to detect both load and dump

In [41]:
import dataloader
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import geopy.distance
from schemas import Trip
import datetime as dt
import ipyleaflet as L
from ipywidgets import Layout
from datetime import timedelta
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import datetime
from tqdm import tqdm

In [42]:
#Day we want to look at
trip = dataloader.TripsLoader('03-11-2022')

In [43]:
#K-means
#Get all dump positions for a given day
trip_day_before = dataloader.TripsLoader('03-10-2022')
all_load_positions_for_day_before = []
all_dump_positions_for_day_before = []
for machine_number in trip_day_before._machines.keys():
    temp_machine = trip_day_before._machines[machine_number]
    if temp_machine.machine_type == 'Truck':
        all_load_positions_for_day_before.append([trip.load_latlon for trip in temp_machine.trips])
        all_dump_positions_for_day_before.append([trip.dump_latlon for trip in temp_machine.trips])

all_load_positions_for_day_before =  [item for sublist in all_load_positions_for_day_before for item in sublist]
all_dump_positions_for_day_before = [item for sublist in all_dump_positions_for_day_before for item in sublist]

# Assuming 'coordinates' is list of tuples [(lat1, lon1), (lat2, lon2), ...]
load_coordinates_array = np.array(all_load_positions_for_day_before)
dump_coordinates_array = np.array(all_dump_positions_for_day_before)

Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  but expected ndarray.
Could not add row, row was type:  <class 'float'>  

In [44]:
# Set the optimal K value you found from the elbow method and silhouette score analysis
optimal_K = 50

# Fit the KMeans model with the optimal K value
load_kmeans = KMeans(n_clusters=optimal_K, random_state=42, n_init='auto')
dump_kmeans = KMeans(n_clusters=optimal_K, random_state=42, n_init='auto')
load_kmeans.fit(load_coordinates_array)
dump_kmeans.fit(dump_coordinates_array)
# Get the coordinates of the cluster centers for the optimal K value
load_cluster_centers = load_kmeans.cluster_centers_
dump_cluster_centers = dump_kmeans.cluster_centers_

In [45]:
#Machine we want to look at
pred_dict_error = {}
for machine_nb in tqdm(trip._machines.keys()):
    some_machine = trip._machines[machine_nb]
    
    if some_machine.machine_type == 'Truck':

        predicted_load_points = []
        predicted_load_times = []
        predicted_dump_points = []
        predicted_dump_times = []
        
        day_speeds = []
        day_dists = []
        day_times = []
        #entering_working_area = []
        #exiting_working_area = []

        nb_of_positions_for_vector = 30
        list_of_inner_prods = []

        all_pos = [trips.positions for trips in some_machine.trips]
        all_pos = [item for sublist in all_pos for item in sublist]

        #Now have all positions with timestamps in one liste
        #Can get actual points and times of loading
        actual_load_points = [trips.load_latlon for trips in some_machine.trips]
        actual_load_times = [trips.positions[0].timestamp for trips in some_machine.trips]
        actual_dump_points = [trips.dump_latlon for trips in some_machine.trips]
        actual_dump_times = []
        for t in some_machine.trips: #Not pretty
            temp_dump_laton = t.dump_latlon
            for position in t.positions:
                if temp_dump_laton == (position.lat, position.lon):
                    actual_dump_times.append(position.timestamp)
                    break

        #First registered point is necessarily loading
        predicted_load_points.append((all_pos[0].lat,all_pos[0].lon))
        predicted_load_times.append(all_pos[0].timestamp)
        predicting_load = False #False meaning that next next prediction will be dump

        in_load_working_area = False #Probably true, since first position is when loading
        in_dump_working_area = False #Maybe false, since first position is when loading

        for coord in load_cluster_centers: #But verify with created clusters of load points from day before
                if geopy.distance.geodesic(coord, (all_pos[0].lat, all_pos[0].lon)).m < 30:
                    in_load_working_area = True

        for coord in dump_cluster_centers: #But verify with created clusters of load points from day before
                if geopy.distance.geodesic(coord, (all_pos[0].lat, all_pos[0].lon)).m < 30:
                    in_dump_working_area = True
            
        meters_since_last_activity = 0

        for i in range(1,len(all_pos[1:])):
            
            current_pos = all_pos[i]
            prev_pos = all_pos[i-1]
            
            #Seconds passed since last timestamp
            seconds_gone = (current_pos.timestamp.to_pydatetime()-prev_pos.timestamp.to_pydatetime()).total_seconds()

            if seconds_gone > 0:

                #Meters driven since last timestamp
                meters_driven = geopy.distance.geodesic((current_pos.lat, current_pos.lon), (prev_pos.lat, prev_pos.lon)).m
                meters_since_last_activity += meters_driven

                if i > 1+nb_of_positions_for_vector: #Create vector for computing reverse of vehicle
                    current_vector = [all_pos[i].lat-all_pos[i-nb_of_positions_for_vector].lat,all_pos[i].lon-all_pos[i-nb_of_positions_for_vector].lon]
                    prev_vector = [all_pos[i-1].lat-all_pos[i-1-nb_of_positions_for_vector].lat,all_pos[i-1].lon-all_pos[i-1-nb_of_positions_for_vector].lon]

                    current_vector_norm = current_vector/np.linalg.norm(current_vector)
                    prev_vector_norm = prev_vector/np.linalg.norm(prev_vector)
                    inner_product = np.inner(current_vector_norm,prev_vector_norm)
                    list_of_inner_prods.append(inner_product)
                else:
                    list_of_inner_prods.append(0)
                
                speed_kmh = (meters_driven/seconds_gone)*3.6
                day_speeds.append(speed_kmh)
                day_dists.append(meters_driven/1000)
                day_times.append(current_pos.timestamp)

                currently_in_load_working_area = False
                for coord in load_cluster_centers:
                    if geopy.distance.geodesic(coord, (current_pos.lat, current_pos.lon)).m < 30:
                        currently_in_load_working_area = True
                
                currently_in_dump_working_area = False
                for coord in dump_cluster_centers:
                    if geopy.distance.geodesic(coord, (current_pos.lat, current_pos.lon)).m < 30:
                        currently_in_dump_working_area = True
                    

                #Two below blocks not really necessary if we are not to plot when the leave or enter work area    
                if not in_load_working_area and currently_in_load_working_area:
                    #entering_working_area.append(current_pos.timestamp)
                    in_load_working_area = True
                elif in_load_working_area and not currently_in_load_working_area:
                    #exiting_working_area.append(current_pos.timestamp)
                    in_load_working_area = False

                if not in_dump_working_area and currently_in_dump_working_area:
                    #entering_working_area.append(current_pos.timestamp)
                    in_dump_working_area = True
                elif in_dump_working_area and not currently_in_dump_working_area:
                    #exiting_working_area.append(current_pos.timestamp)
                    in_dump_working_area = False

                #Logic for predicting loading point
                if speed_kmh < 5 and meters_since_last_activity>200:
                    if predicting_load:
                        if in_load_working_area:
                            last_min_start = current_pos.timestamp-timedelta(minutes=2)
                            index_start_minute = None
                            for i, ts in enumerate(day_times):
                                if ts >= last_min_start:
                                    index_start_minute = i
                                    break
                            sum_over_last_minute = np.sum(day_speeds[index_start_minute:])
                            
                            if sum_over_last_minute < 50:
                                predicted_load_points.append((current_pos.lat, current_pos.lon))
                                predicted_load_times.append(current_pos.timestamp)
                                
                                #Have now predicted a load
                                predicting_load = False
                                meters_since_last_activity = 0

                    else: #Predicting dump
                        if in_dump_working_area:
                            last_min_start = current_pos.timestamp-timedelta(minutes=1)
                            index_start_minute = None
                            for i, ts in enumerate(day_times):
                                if ts >= last_min_start:
                                    index_start_minute = i
                                    break
                            sum_over_last_minute = np.sum(day_speeds[index_start_minute:])
                            
                            if sum_over_last_minute < 500 and list_of_inner_prods[-1] < 0.98: #Not ideal, want to instead pick the best among times, not just the first viable option, but difficult with live tracking.
                                predicted_dump_points.append((current_pos.lat, current_pos.lon))
                                predicted_dump_times.append(current_pos.timestamp)
                                
                                #Have now predicted a dump
                                predicting_load = True
                                meters_since_last_activity = 0

                     
        
                
        # Plots, not wanted when a lot of data
        # Create subplots with 2 rows and 1 column
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.1)

        # Add the first line plot to the first subplot
        fig.add_trace(go.Scatter(x=day_times, y=day_speeds, mode='lines', name='Speed'), row=1, col=1)
        fig.add_trace(go.Scatter(x=actual_load_times, y=[0 for a in actual_load_times], mode='markers', marker=dict(symbol='cross', size=10, color='red'), name='Load actual'), row=1, col=1)
        fig.add_trace(go.Scatter(x=predicted_load_times, y=[0 for p in predicted_load_times], mode='markers', marker=dict(symbol='star', size=10, color='red'), name='Load predicted'), row=1, col=1)
        fig.add_trace(go.Scatter(x=actual_dump_times, y=[0 for a in actual_dump_times], mode='markers', marker=dict(symbol='cross', size=10, color='green'), name='Dump actual'), row=1, col=1)
        fig.add_trace(go.Scatter(x=predicted_dump_times, y=[0 for p in predicted_dump_times], mode='markers', marker=dict(symbol='star', size=10, color='green'), name='Dump predicted'), row=1, col=1)

        # Add the second line plot to the second subplot
        fig.add_trace(go.Scatter(x=day_times, y=np.cumsum(day_dists), mode='lines', name='Cumulative distance'), row=2, col=1)
        fig.add_trace(go.Scatter(x=actual_load_times, y=[0 for a in actual_load_times], mode='markers', marker=dict(symbol='cross', size=10, color='red'), name='Load actual'), row=2, col=1)
        fig.add_trace(go.Scatter(x=predicted_load_times, y=[0 for p in predicted_load_times], mode='markers', marker=dict(symbol='star', size=10, color='red'), name='Load predicted'), row=2, col=1)
        fig.add_trace(go.Scatter(x=actual_dump_times, y=[0 for a in actual_dump_times], mode='markers', marker=dict(symbol='cross', size=10, color='green'), name='Dump actual'), row=1, col=1)
        fig.add_trace(go.Scatter(x=predicted_dump_times, y=[0 for p in predicted_dump_times], mode='markers', marker=dict(symbol='star', size=10, color='green'), name='Dump predicted'), row=1, col=1)

        # Add the third line plot
        fig.add_trace(go.Scatter(x=day_times, y=list_of_inner_prods, mode='lines', name='Inner product of vectors'), row=3, col=1)

        # Update layout settings for both subplots
        fig.update_layout(title=str('Subplots of Speeds and cumulative distance, machine_id: '+ str(some_machine.machine_id)),
                        xaxis_title='Timestamp',
                        showlegend=True)

        fig.show()

  0%|          | 0/23 [00:00<?, ?it/s]

  4%|▍         | 1/23 [03:34<1:18:44, 214.73s/it]


KeyboardInterrupt: 

In [ ]:
# Fredag

# Gjøre om på kriterier for dump, siden første tilfelle ikke blir plukket opp.
# Øke fart? Eller noeeeee.